In [4]:
# ****** Unir TFM ***
# *** Valorizacion ***
# Javier Ulloa ***
# ***************

In [1]:
import os
import csv
import nltk
import time
import pandas as pd
import requests
from datetime import datetime, timedelta
from PyPDF2 import PdfReader
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from gensim import corpora, models
from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions
import os
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim import corpora, models
import nltk


##Codigo Adicional agregado
import sys

# Configuración del entorno del notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.dirname(notebook_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from config import PDF_DIRECTORY_BOW, CSV_DIRECTORY_BOW

In [3]:
# descarga nltk
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)
nltk.download("omw", quiet=True)

# Credenciales de acceso a SQL server
account_name = "account_name"
account_key = "account_key"
container_name = "discursos"


# Creacion de un cliente de interaccion con blob storage
connect_str = (
    "DefaultEndpointsProtocol=https;AccountName="
    + account_name
    + ";AccountKey="
    + account_key
    + ";EndpointSuffix=core.windows.net"
)
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# conexion del container
container_client = blob_service_client.get_container_client(container_name)

# obtener una lista de contenedores
blob_list = []
for blob_i in container_client.list_blobs():
    blob_list.append(blob_i.name)
# print(blob_list)


# Ruta del directorio donde se descargaron los archivos PDF
pdf_directory = PDF_DIRECTORY_BOW

sas_urls = []
# generate a shared access signiture for files and load them into Python
for blob_i in blob_list:
    # generate a shared access signature for each blob file
    sas_i = generate_blob_sas(
        account_name=account_name,
        container_name=container_name,
        blob_name=blob_i,
        account_key=account_key,
        permission=BlobSasPermissions(read=True),
        expiry=datetime.utcnow() + timedelta(hours=1),
    )

    sas_url = (
        "https://"
        + account_name
        + ".blob.core.windows.net/"
        + container_name
        + "/"
        + blob_i
        + "?"
        + sas_i
    )
    sas_urls.append(sas_url)

# print(sas_urls)

# Lista de archivos PDF
pdf_files = []

# Extrae el país, el presidente y el año del nombre del archivo
for sas_url in sas_urls:
    nombre = sas_url.split(".")[4]
    nombre_sin = nombre.split("/")[-1]
    nombre_archivo = nombre_sin.split(".")[0]
    president = sas_url.split("/")[-3]
    country = sas_url.split("/")[-4]
    pais = nombre_sin.split("_")[0]
    ano = nombre_sin.split("_")[1]
    nombre = nombre_sin.split("_")[2]
    apellido = nombre_archivo.split("_")[3]

    # Genera el nombre del archivo PDF
    filename = f"{pais}_{ano}_{nombre}_{apellido}.pdf"

    # Descarga el archivo PDF
    response = requests.get(sas_url)
    with open(os.path.join(pdf_directory, filename), "wb") as f:
        f.write(response.content)

    # Agrega el archivo PDF a la lista de archivos descargados
    pdf_files.append(filename)

    year = ""
    nombre_archivo = ""
    president = ""
    country = ""
    pais = ""
    ano = ""
    nombre = ""
    apellido = ""

# Imprime la lista de archivos PDF
# print(pdf_files)

KeyboardInterrupt: 

In [7]:
### 1era Parte

In [4]:
import nltk

nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...


True

In [5]:
# ##########################
# Polarizacion
# ##########################
# Palabras mas frecuentes
###########################


import os
import pandas as pd
from datetime import datetime
from PyPDF2 import PdfReader
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
import nltk
import spacy
from collections import Counter


####################
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)
nltk.download("wordnet")
nltk.download("treebank")
nltk.download("maxent_ne_chunker")

# Archivos descargados de la nube
carpeta_pdf = PDF_DIRECTORY_BOW
####################

# Descargar el modelo en español para spaCy
spacy.cli.download("es_core_news_sm")

# Inicializar el lematizador de NLTK
lemmatizer = nltk.stem.WordNetLemmatizer()

# Diccionario para almacenar palabras frecuentes por oración
# sin stopwords
palabras_frecuentes = {}
# con stopwords
palabras_frecuentes2 = {}


# Recueprar texto de los PDFs
def procesar_oraciones(ruta_pdf):
    with open(ruta_pdf, "rb") as archivo_pdf:
        cont_pdf = PdfReader(archivo_pdf)
        texto = ""
        for i in range(len(cont_pdf.pages)):
            pagina = cont_pdf.pages[i]
            texto += pagina.extract_text()
        oraciones = sent_tokenize(texto)
    return oraciones


# Polarizacion de los textos
def clasificar_texto(oracion):
    termometro = SentimentIntensityAnalyzer()
    temperatura = termometro.polarity_scores(oracion)["compound"]
    if temperatura >= 0.05:
        resultado = "Positivo"
    elif temperatura <= -0.05:
        resultado = "Negativo"
    else:
        resultado = "Neutral"
    return temperatura, resultado


# Proceso de los discursos presidenciales
def procesar_archivos_en_carpeta(carpeta_pdf):
    campos = [
        "Fecha",
        "Pais",
        "Presidente",
        "Orden",
        "Discurso",
        "Oracion",
        "TextoProcesado",
        "PalabrasEliminadas",
        "Clasificacion",
        "Valoracion",
        "Clasificacion2",
        "Valoracion2",
        "PalabrasFrecuentes",
        "PalabrasFrecuentes2",
    ]
    data_list = []

    for archivo_pdf in os.listdir(carpeta_pdf):
        if archivo_pdf.endswith(".pdf"):
            ruta_pdf = os.path.join(carpeta_pdf, archivo_pdf)
            nombre_archivo = os.path.splitext(os.path.basename(ruta_pdf))[0]
            elementos = nombre_archivo.split("_")
            pais = elementos[0]
            anio = int(elementos[1])
            nombre_presidente = " ".join(elementos[2:])
            oraciones = procesar_oraciones(ruta_pdf)

            for i, oracion in enumerate(oraciones, start=1):
                oracion_con_formato = oracion.replace("\n", "").replace("'", "").strip()
                # Calcula temperatura y valoración con la columna "Oracion"
                valoracion, temperatura = clasificar_texto(oracion_con_formato)
                stop_words = set(stopwords.words("spanish"))
                word_tokens = word_tokenize(oracion.lower())
                filtered_sentence = [
                    word
                    for word in word_tokens
                    if word.isalnum() and word not in stop_words
                ]
                texto_procesado = " ".join(filtered_sentence)
                # Palabras_a_eliminar
                # palabras_a_eliminar = ['S','s', 'si', 'pr', 'ón', 'és', 'c', 're', 'po', 'r', 'ta', 'pues','si', 'on', 'na', 'co', 're', 'blo','5','pues', 'e']

                # Almacena las palabras eliminadas (stopwords)
                palabras_eliminadas = [
                    word for word in word_tokens if word.lower() in stop_words
                ]
                palabras_eliminadas_str = "{" + ", ".join(palabras_eliminadas) + "}"
                # palabras_eliminadas_str = '{' + ', '.join(palabras_a_eliminar) + '}'
                # Realiza el análisis de sentimientos para la columna "TextoProcesado"
                valoracion2, temperatura2 = clasificar_texto(texto_procesado)
                # Lematización e Identificación de palabras frecuentes
                nlp = spacy.load("es_core_news_sm")
                lematizados_spacy = [token.lemma_ for token in nlp(oracion_con_formato)]
                lematizados_spacy2 = [token.lemma_ for token in nlp(texto_procesado)]

                # Contar la frecuencia de cada palabra lematizada
                frecuencia_palabras = Counter(lematizados_spacy)
                frecuencia_palabras2 = Counter(lematizados_spacy2)

                # Obtener las tres palabras más frecuentes
                top_3_palabras = frecuencia_palabras.most_common(3)
                top_3_palabras2 = frecuencia_palabras2.most_common(3)

                palabras_frecuentes[oracion_con_formato] = [
                    palabra for palabra, _ in top_3_palabras
                ]
                palabras_frecuentes2[texto_procesado] = [
                    palabra for palabra, _ in top_3_palabras2
                ]

                valorizaciones = {
                    "Fecha": anio,
                    "Pais": pais,
                    "Presidente": nombre_presidente,
                    "Orden": i,
                    "Discurso": nombre_archivo + ".pdf",
                    "Oracion": oracion_con_formato,
                    "TextoProcesado": texto_procesado,
                    "PalabrasEliminadas": palabras_eliminadas_str,
                    "Clasificacion": temperatura,
                    "Valoracion": valoracion,
                    "Clasificacion2": temperatura2,
                    "Valoracion2": valoracion2,
                    "PalabrasFrecuentes": palabras_frecuentes[oracion_con_formato],
                    "PalabrasFrecuentes2": palabras_frecuentes2[texto_procesado],
                }

                data_list.append(valorizaciones)

    df = pd.DataFrame(data_list, columns=campos)
    return df


# Procesa los archivos en la carpeta y crea el DataFrame
data = procesar_archivos_en_carpeta(carpeta_pdf)

# Guarda el DataFrame en un archivo CSV
csv_output_path = CSV_DIRECTORY_BOW + "\speech_evaluation.csv"  # ojo modifcicacion
data.to_csv(csv_output_path, index=False)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
